# Lib

In [24]:
%load_ext autoreload
%autoreload 2

import pandas as pd

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 10000)


import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sys
import os
import glob

if sys.platform=='win32':
    sys.path.insert(0,".\..\src")
        
elif sys.platform=='linux':
    sys.path.insert(0,"./../src")


from sklearn.model_selection import train_test_split
from utils import utils_ml, parameters


from ipywidgets import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
if sys.platform=='win32':
    !dir .\
        
elif sys.platform=='linux':
    !ls ./

 O volume na unidade C � OS
 O N�mero de S�rie do Volume � C498-8BF5

 Pasta de c:\Users\caanp\OneDrive\Documents\repositories\ML-anti-fraud\notebooks

18/11/2022  19:16    <DIR>          .
23/11/2022  00:25    <DIR>          ..
10/09/2022  23:28    <DIR>          .ipynb_checkpoints
21/11/2022  23:51         1.007.073 1.EA.ipynb
21/11/2022  23:43            70.541 2.Feature_engineering.ipynb
22/11/2022  00:00            32.520 3.Stability_var.ipynb
21/11/2022  22:52           128.728 4.Modeling.ipynb
21/11/2022  19:26    <DIR>          catboost_info
               4 arquivo(s)      1.238.862 bytes
               4 pasta(s)   360.389.447.680 bytes dispon�veis


# Reading

In [26]:
## para saber en que directório uno se encuentra
#
if sys.platform=='win32':
    !echo %cd%
        
elif sys.platform=='linux':
    !pwd

c:\Users\caanp\OneDrive\Documents\repositories\ML-anti-fraud\notebooks


In [27]:
if sys.platform=='win32':
    df = pd.read_csv('.\..\data\dados.csv', sep=',', decimal=".")
        
elif sys.platform=='linux':
    df = pd.read_csv('./../data/dados.csv', sep=',', decimal=".")


## dropping column "o" becaise it has 78% missing. and this is a boolean columns
# df.drop("o", axis=1, inplace=True)

### Renaming and formatting columns

In [28]:
df.rename(columns={"a":"a_int",
                    "p":"p_boolean","n":'n_boolean',
                     "k":"k_float","e":"e_float","h":"h_float","l":"l_float","f":"f_float","m":"m_float","m":"m_float","b":"b_float","c":"c_float","d":"d_float","o":"o_obj",
                     'i': 'produto','j':'categoria_produto','g':'pais'}
       , inplace = True)

## Transform categorical features
df['n_boolean'] = df['n_boolean'].astype("category")
df['p_boolean'] = df['p_boolean'].astype("category")


## creating columns os date and hour
df['data'] = pd.to_datetime( df['fecha'].apply(lambda x: x[:10]) )
df['hora'] = df['fecha'].apply(lambda x: x[10:])


# Split train, test and validation datasets

In [29]:
seman = df.groupby("data").agg( 


    total_um = ("fraude",lambda x: sum(x) ),
    total_zero = ("fraude",lambda x: sum( (x+1)%2 ) ),
    total = ("fraude","count" )
    
     )


seman['%_cusum'] = (seman['total']/seman['total'].sum()).cumsum()

seman.reset_index()

data  total_um  total_zero  total   %_cusum
0  2020-03-08       124        2714   2838  0.018920
1  2020-03-09       197        3976   4173  0.046740
2  2020-03-10       187        3918   4105  0.074107
3  2020-03-11       203        3696   3899  0.100100
4  2020-03-12       181        3606   3787  0.125347
5  2020-03-13       162        3067   3229  0.146873
6  2020-03-14       135        2425   2560  0.163940
7  2020-03-15       119        2651   2770  0.182407
8  2020-03-16       180        4022   4202  0.210420
9  2020-03-17       201        4079   4280  0.238953
10 2020-03-18       182        3859   4041  0.265893
11 2020-03-19       193        3706   3899  0.291887
12 2020-03-20       153        3163   3316  0.313993
13 2020-03-21       109        2523   2632  0.331540
14 2020-03-22       131        2698   2829  0.350400
15 2020-03-23       200        3796   3996  0.377040
16 2020-03-24       220        3648   3868  0.402827
17 2020-03-25       155        3247   3402  0.425507
18 2020-03-26       183        2658   2841  0.444447
19 2020-03-27       130        1897   2027  0.457960
20 2020-03-28       125        1847   1972  0.471107
21 2020-03-29       129        1834   1963  0.484193
22 2020-03-30       152        2212   2364  0.499953
23 2020-03-31       126        1842   1968  0.513073
24 2020-04-01       166        2416   2582  0.530287
25 2020-04-02       160        2581   2741  0.548560
26 2020-04-03       129        2479   2608  0.565947
27 2020-04-04       137        2437   2574  0.583107
28 2020-04-05       138        2459   2597  0.600420
29 2020-04-06       183        3424   3607  0.624467
30 2020-04-07       230        3343   3573  0.648287
31 2020-04-08       199        3575   3774  0.673447
32 2020-04-09       223        3741   3964  0.699873
33 2020-04-10       204        3281   3485  0.723107
34 2020-04-11       139        2959   3098  0.743760
35 2020-04-12       144        2798   2942  0.763373
36 2020-04-13       131        2087   2218  0.778160
37 2020-04-14       202        4088   4290  0.806760
38 2020-04-15       100        4373   4473  0.836580
39 2020-04-16       198        3728   3926  0.862753
40 2020-04-17       160        3444   3604  0.886780
41 2020-04-18       140        3270   3410  0.909513
42 2020-04-19       156        3255   3411  0.932253
43 2020-04-20       258        5056   5314  0.967680
44 2020-04-21       226        4622   4848  1.000000

>> Splitting dataset in validation and train-test samples

In [53]:
n = 5
## Selecting the train and test to model (aprox 86% of full dataset)
datas_train_test = np.sort( df['data'].unique() )[:-n]
df_train_test = df[df['data'].isin(datas_train_test)]
## selecting the last n dates to validate (aprox 14% of full dataset)
datas_validation = np.sort( df['data'].unique() )[::-1][:n]
df_validation = df[df['data'].isin(datas_validation)]

In [57]:
df_train_test.shape[0]/df.shape[0], df_validation.shape[0]/df.shape[0]

(0.8627533333333334, 0.13724666666666666)

>> Splitting the sample to contruct the WoE transformation

In [32]:
X = df_train_test.drop(['fraude'], axis=1)
y = df_train_test['fraude']

## Spliting the sample
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.3, random_state=0, stratify=y )

## Creating the label column to save
X_train['fraude'] = y_train
X_test['fraude'] = y_test

## Saving 
X_train.to_parquet(f"{parameters.data_path}\\train_dataset.parquet", index=False)
X_test.to_parquet(f"{parameters.data_path}\\test_dataset.parquet", index=False)
df_validation.to_parquet(f"{parameters.data_path}\\validation_dataset.parquet", index=False)

## WoE transformation pipeline

In [33]:
cats_cols = utils_ml.list_subset_words( df_train_test.columns.tolist() , ["bool","cat",'obj','pais'])
numeric_cols = [x for x in df_train_test.columns.tolist() if df_train_test[x].dtype.name!="object" and df_train_test[x].dtype.name!="category" and x!="fraude" ]

features = cats_cols + numeric_cols


features

['n_boolean',
 'p_boolean',
 'categoria_produto',
 'o_obj',
 'pais',
 'a_int',
 'b_float',
 'c_float',
 'd_float',
 'e_float',
 'f_float',
 'h_float',
 'k_float',
 'l_float',
 'm_float',
 'monto',
 'score',
 'data']

### Encoding

In [34]:
dff = df_train_test.copy()

for variable in features:

    label = "fraude"
    dtype_ = 'categorical' if dff[variable].dtype.name=='object' or dff[variable].dtype.name=='category' else 'numerical'
    monotonic = True
    save_path_pkl = '.\..\src\\features\\'

    ## creating the WoE transformation and saving in pickle object.
    dff = utils_ml.encode_woe_var(dff, variable, label, dtype_, monotonic = True, save_path_pkl = save_path_pkl )

In [35]:
features_bins_woe = utils_ml.list_subset_words( dff.columns.tolist(), ["bin"])
features_bins_woe

['n_boolean_bins',
 'p_boolean_bins',
 'categoria_produto_bins',
 'o_obj_bins',
 'pais_bins',
 'a_int_bins',
 'b_float_bins',
 'c_float_bins',
 'd_float_bins',
 'e_float_bins',
 'f_float_bins',
 'h_float_bins',
 'k_float_bins',
 'l_float_bins',
 'm_float_bins',
 'monto_bins',
 'score_bins',
 'data_bins']

In [36]:
len(features_bins_woe)

18

# Saving

In [42]:
dff

a_int  b_float    c_float  d_float   e_float  f_float pais  h_float                                            produto categoria_produto   k_float  l_float  m_float n_boolean o_obj p_boolean                fecha   monto  score  fraude       data       hora n_boolean_bins n_boolean_cat  n_boolean_WoE p_boolean_bins p_boolean_cat  p_boolean_WoE categoria_produto_bins                              categoria_produto_cat  categoria_produto_WoE o_obj_bins o_obj_cat  o_obj_WoE pais_bins                                           pais_cat  pais_WoE a_int_bins     a_int_cat  a_int_WoE b_float_bins   b_float_cat  b_float_WoE c_float_bins    c_float_cat  c_float_WoE d_float_bins     d_float_cat  d_float_WoE e_float_bins   e_float_cat  e_float_WoE f_float_bins      f_float_cat  f_float_WoE h_float_bins     h_float_cat  h_float_WoE k_float_bins   k_float_cat  k_float_WoE l_float_bins         l_float_cat  l_float_WoE m_float_bins       m_float_cat  m_float_WoE monto_bins        monto_cat  monto_WoE score_bins       score_cat  score_WoE data_bins                                          data_cat  data_WoE
0           4   0.7685   94436.24     20.0  0.444828      1.0   BR        5  Máquininha Corta Barba Cabelo Peito Perna Pelo...       cat_8d714cd  0.883598    240.0    102.0         1   NaN         N  2020-03-27 11:51:16    5.64     66       0 2020-03-27   11:51:16              0           [1]      -0.288024              1         ['N']       0.448404                      8  ['cat_1082639' 'cat_9136c31' 'cat_cafaf9c' 'ca...               0.411709         -1   Missing   0.931027         1  ['BR' 'ES' 'PT' 'CL' 'FR' 'NZ' 'DE' 'EC' 'KR' ...  0.102526          1   [3.50, inf)  -0.120015            3  [0.75, 0.82)    -0.130450            1  [652.62, inf)    -0.080351            4  [19.50, 32.50)    -0.225977            2   [0.18, inf)    -0.186634            1     [0.50, 1.50)     0.323030            1    [0.50, 5.50)     0.038941            4   [0.63, inf)     0.028275            1    [144.50, 339.50)     0.604851            4   [74.50, 153.50)    -0.031594          1    [5.24, 13.68)  -0.105166          1  [65.50, 71.50)  -0.242557         0                    (-inf, 1586520031053742080.00)  0.031516
1           4   0.7550    9258.50      1.0  0.000000     33.0   BR        0  Avental Descartavel Manga Longa  - 50 Un. Tnt ...       cat_64b574b  0.376019   4008.0      0.0         1     Y         N  2020-04-15 19:58:08  124.71     72       0 2020-04-15   19:58:08              0           [1]      -0.288024              1         ['N']       0.448404                      1  ['cat_ab485b4' 'cat_803df0e' 'cat_4c2544e' 'ca...              -0.996546          0     ['Y']   0.269846         1  ['BR' 'ES' 'PT' 'CL' 'FR' 'NZ' 'DE' 'EC' 'KR' ...  0.102526          1   [3.50, inf)  -0.120015            3  [0.75, 0.82)    -0.130450            1  [652.62, inf)    -0.080351            0    (-inf, 1.50)     0.610451            0  (-inf, 0.07)     0.160349            6   [20.50, 41.50)    -0.796174            0    (-inf, 0.50)     0.658319            1  [0.06, 0.42)    -0.030524            9  [3572.50, 4028.50)    -0.732382            0      (-inf, 1.50)     0.930277          4  [53.80, 147.71)   0.208065          2  [71.50, 81.50)   0.186675         2                     [1586865621302247424.00, inf) -0.293590
2           4   0.7455  242549.09      3.0  0.000000     19.0   AR       23  Bicicleta Mountain Fire Bird Rodado 29 Alumini...       cat_e9110c5  0.516368   1779.0     77.0         1   NaN         N  2020-03-25 18:13:38  339.32     95       0 2020-03-25   18:13:38              0           [1]      -0.288024              1         ['N']       0.448404                      2  ['cat_100ac18' 'cat_16839f1' 'cat_734dded' 'ca...              -0.580418         -1   Missing   0.931027         0  ['AD' 'HN' 'IN' 'IT' 'JP' 'LB' 'NI' 'NO' 'PA' ... -0.388054          1   [3.50, inf)  -0.120015            2  [0.72, 0.75)    -0.144167            1  [652.62, inf)    -0.080

In [43]:
## Saving 
dff.to_parquet(f"{parameters.data_path}\\train_test_dataset_tranformed.parquet", index=False)

# WoE to train dataset

### Train dataset

In [50]:
@interact

def display_metrics(variable = features ):

    label = "fraude"
    dtype_ = 'categorical' if X_train[variable].dtype.name=='object' or X_train[variable].dtype.name=='category'  else 'numerical'

    display(utils_ml.WOE_IV(df = X_train , variable = variable, dtype_ = dtype_ , label = label, monotonic= True))

interactive(children=(Dropdown(description='variable', options=('n_boolean', 'p_boolean', 'categoria_produto',…

### Full dataset

In [49]:
@interact

def display_metrics(variable = features ):

    label = "fraude"
    dtype_ = 'categorical' if df[variable].dtype.name=='object' or df[variable].dtype.name=='category'  else 'numerical'

    display(utils_ml.WOE_IV(df = df , variable = variable, dtype_ = dtype_ , label = label, monotonic= True))

interactive(children=(Dropdown(description='variable', options=('n_boolean', 'p_boolean', 'categoria_produto',…

# Requirements

In [96]:
path_root = parameters.root
!pip list --format=freeze > $path_root/requirements.txt